In [15]:
#Basado en: https://www.eduonix.com/dashboard/complete-guide-to-machine-learning-using-python
import pandas as pd
import numpy as np
import rake_nltk as rake

# conda install -c conda-forge rake_nltk
# conda install -c conda-forge scikit-surprise

r = rake.Rake()
movies = pd.read_csv('movies_metadata.csv', low_memory=False)

# print(movies["vote_average"].value_counts())

m_count = movies['vote_count'].quantile(0.85)

movies = movies[movies['vote_count'] > m_count]
movies = movies[movies['overview'].notnull()]


82.0


In [19]:
import rake_nltk as rake
import sys
r = rake.Rake()

movies["keywords"] = ""

for index, row in movies.iterrows():
    review = row["overview"]

    r.extract_keywords_from_text(review)
    
    keyDegrees = r.get_word_degrees()
    
    keyDegrees = {k: kw for k, kw in keyDegrees.items() if kw > 1} # kw = keyword
    movies.loc[index,"keywords"] = " ".join(keyDegrees.keys())

reviews = movies[["original_title","keywords"]]
# titleInd = {i: movies.loc[i]["original_title"] for i in movies.index}

reviews = reviews.set_index('original_title')

In [23]:
## Memory-model (Cosine Distance)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import sys

indices = pd.Series(movies.index, index=movies['original_title']).drop_duplicates()
indiceList = reviews.index.tolist() # Indexes are not consistent range from 0 to N. 

count = TfidfVectorizer()
count_matrix = count.fit_transform(reviews['keywords'])


cosine_sim = cosine_similarity(count_matrix, count_matrix)

idx = indices["Toy Story"]
recommendations = list(enumerate((cosine_sim[idx])))

recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)[0:11]
# movie_indices = [i[0] for i in score_series]

for ind, weight in recommendations:
    print(indiceList[ind],weight)

Toy Story 1.0
Toy Story 2 0.2430083118646356
The Last Airbender 0.12530953375573964
Hawaiian Vacation 0.12356195033784233
The War of the Roses 0.12312612014134833
The Wild 0.12139800914765503
Mean Machine 0.11314896159638423
Halloweentown 0.11308456899610897
Hunt for the Wilderpeople 0.10810572830469659
Bring It On Again 0.10544730248314262
Rebel Without a Cause 0.10484761180949846


In [3]:
from surprise import SVD, evaluate, Dataset, Reader
from surprise.model_selection import cross_validate
from surprise import NMF
import pandas as pd

import sys

ratings = pd.read_csv('ratings_small.csv', low_memory=False)
ratings = ratings.drop(['timestamp'],axis=1)
print(ratings)
sys.exit()
reader = Reader(rating_scale=(0.5,5.0))

ratings = Dataset.load_from_df(ratings, reader)
ratings.split(n_folds=5)

# svd
svd = SVD()
nmf = NMF()
# evaluate(svd, ratings, measures=['RMSE'])
cross_validate(nmf, ratings, measures=['RMSE', 'MAE'], cv=5, verbose=True)



        userId  movieId  rating
0            1       31     2.5
1            1     1029     3.0
2            1     1061     3.0
3            1     1129     2.0
4            1     1172     4.0
5            1     1263     2.0
6            1     1287     2.0
7            1     1293     2.0
8            1     1339     3.5
9            1     1343     2.0
10           1     1371     2.5
11           1     1405     1.0
12           1     1953     4.0
13           1     2105     4.0
14           1     2150     3.0
15           1     2193     2.0
16           1     2294     2.0
17           1     2455     2.5
18           1     2968     1.0
19           1     3671     3.0
20           2       10     4.0
21           2       17     5.0
22           2       39     5.0
23           2       47     4.0
24           2       50     4.0
25           2       52     3.0
26           2       62     3.0
27           2      110     4.0
28           2      144     3.0
29           2      150     5.0
...     

SystemExit: 

/Users/arayi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
